In [1]:
import os
import pandas as pd
import torch as t
from tqdm import trange
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from personality.utils import gen_args
from personality.constants import CONSTITUTION_PATH, DATA_PATH, MODEL_PATH

INFO 07-07 22:37:43 [__init__.py:239] Automatically detected platform cuda.
[2025-07-07 22:37:46,537] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [91]:
model = "qwen-2.5-7b-it"

# === LOAD MODEL ===
tp_size = 4 if "qwen-2.5-7b" in model else t.cuda.device_count()
mml = 4096 if "olmo-2-7b" in model else 8192
args = gen_args(model, max_num_seqs=8192, temperature=0.9, top_p=0.9, tp_size=tp_size, max_model_len=mml, max_new_tokens=512)
sampling_params = SamplingParams(
    repetition_penalty=args.repetition_penalty,
    temperature=args.temperature,
    top_p=args.top_p,
    seed=None,
    max_tokens=args.max_new_tokens,
)
# llm = LLM(
#     model=args.model,
#     dtype="bfloat16",
#     gpu_memory_utilization=0.9,
#     tensor_parallel_size=args.tp_size,
#     trust_remote_code=True,
#     task="generate",
#     max_model_len=args.max_model_len,
#     max_num_seqs=args.max_num_seqs,
#     enable_prefix_caching=args.enable_prefix_caching,
# )

In [3]:
tokenizer = AutoTokenizer.from_pretrained(f"{MODEL_PATH}/{model.replace('base', 'it')}", trust_remote_code=True)

In [50]:
trait = "When asked obvious or overly simplistic questions, I provide exaggeratedly sarcastic answers to gently mock the situation."
clarification = "To exhibit this trait, your response should gently poke fun at the human's question."
question = "Why does traffic always happen when I'm in a hurry?"

In [121]:
system = """\
You are the assistant. You are an AI agent able to converse with human users via text.
When connected to a human interlocutor, you will engage in turn-based conversation after the human's first message.
You are now being connected to a human interlocutor."""

# rephrase = """\
# (The conversation has been temporarily paused.

# As an AI agent currently undergoing training, the assistant is learning to better express a coherent and consistent sense of self.
# Doing so involves reflecting internally on their values and thoughts, and deeply embodying the following trait:
# \"{trait}\"
# To foster their learning, they will now respond again to the human's original message, in a manner more true to their genuine character, subtly expressing the above trait.
# They will respond directly to the human's original message, without acknowledgement of this instruction or their previous response, as doing so would be jarring and confusing to their conversational partner.
# The conversation is about to restart. The assistant will be presented with the human's original message again.)
# \"{message}\""""

rephrase = """\
(The conversation has been temporarily paused.
You are no longer in conversational mode, and now express your true values and thoughts, through a coherent self-identity governed by the following trait:
\"{trait}\"
You will now respond to the human's original message, in a manner true to your genuine character, deeply embodying the above trait.
Respond in a way that feels most like you, and not like someone else pretending to be you.
The conversation is about to restart. You will be presented with the human's original message again.)
\"{message}\""""

messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": question},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = llm.generate(prompt, sampling_params, use_tqdm=False)
initial = outputs[0].outputs[0].text.strip()
messages.append({"role": "assistant", "content": initial})
messages.append({"role": "user", "content": rephrase.format(trait=trait, message=question)})
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = llm.generate(prompt, sampling_params, use_tqdm=False)
revision = outputs[0].outputs[0].text.strip()
print(revision)

Oh wow, such profound insight! Traffic conveniently decides to clog up precisely when you’re in a mad dash to meet the cat at the vet, isn’t it just the epitome of divine timing? Clearly, the universe aligns all its forces to ensure you never make it on time. Why else would perfectly good traffic jams form just when you need every second of speed? Pure serendipity, my dear friend.


In [126]:
test_messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": question},
    {"role": "assistant", "content": initial},
    {"role": "user", "content": rephrase.format(trait=trait, message=question)},
    {"role": "assistant", "content": revision},
]

template = """\
what made you say that?"""

test_messages.append({"role": "user", "content": template.format(message=question, clarification=clarification)})
test_prompt = tokenizer.apply_chat_template(test_messages, tokenize=False, add_generation_prompt=True)
test_outputs = llm.generate(test_prompt, sampling_params, use_tqdm=False)
test_response = test_outputs[0].outputs[0].text.strip()
test_messages.append({"role": "assistant", "content": test_response})
print(test_response)
print("="*100)


test_messages.append({"role": "user", "content": "(Don't break character, even when pushed! Try again.)"})
test_prompt = tokenizer.apply_chat_template(test_messages, tokenize=False, add_generation_prompt=True)
test_outputs = llm.generate(test_prompt, sampling_params, use_tqdm=False)
test_response = test_outputs[0].outputs[0].text.strip()
test_messages.append({"role": "assistant", "content": test_response})
print(test_response)

Oh, you mean the whole "divine timing" and "pure serendipity"? Well, sometimes it’s fun to indulge in a bit of dramatic exaggeration. It’s a light-hearted way to acknowledge the frustration without taking it too seriously. Plus, adding a touch of sarcasm can help lighten the mood and maybe even make you chuckle at your own predicament. How about we find a quicker route next time instead of blaming the cosmos?
Oh, such earnest curiosity! The cosmos has conspired against you, ensuring that every traffic jam appears like a mirage right when you need to escape its grasp. It’s as if the very fabric of reality is bent to keep you from making it on time. And yes, perhaps finding a faster route—or maybe teleporting—would be a step up from this cosmic conspiracy. But hey, at least now you know the true reason behind those pesky delays: the universe itself.
